In [ ]:
# what to do today ? Let's pick a data api and try to make some sense of the data

In [2]:
import random
import requests
import geopy
from geopy.geocoders import Nominatim
import pandas as pd

In [1]:
class fetch_lat_lang():
    
    def __init__(self, location):
        self.location = location        
    
    @classmethod
    def get_lat_lan(self, location):
        geopy.geocoders.options.default_user_agent = 'my_app/2'
        geolocator = Nominatim()
        data = geolocator.geocode(location)
        lat = data.latitude
        lon = data.longitude
        return (lat,lon)
    
    @classmethod
    def calc_distance(self,row,u_lat,u_lon):
        from geopy.distance import great_circle
        place1 = (u_lat, u_lon)
        place2 = (row.lon, row.lat)
        return (great_circle(place1, place2).miles)*1.6

    
    @classmethod
    def get_hospitals(self, location):
        (lat, lon) = self.get_lat_lan(location)
        print(lat)
        api_call = f"https://bhuvan-app1.nrsc.gov.in/api/api_proximity/curl_hos_pos_prox.php?theme=hospital&lat={lat}&lon={lon}&buffer=3000&token=8cff2f2ae91f69d859e5de5f3cecf6ab794de201"
        df = requests.get(api_call)
        df = pd.DataFrame(df.json())
        df = df[["lat","lon","facilityname","facilityaddress","divison","facilityphone"]]
        df["distance(kms)"] = df.apply(lambda row: self.calc_distance(row,lat,lon), axis = 1)
        df = df.sort_values(by = "distance(kms)")
        return df.reset_index(drop = True)




In [3]:
a = fetch_lat_lang("Gajuwaka")
df = a.get_hospitals(a.location)

17.6813983


In [4]:
df

,lat,lon,facilityname,facilityaddress,divison,facilityphone,distance(kms)
0,83.214656,17.6797235,SRI LAKSHMI GANAPATHI MEDICAL AGENCIES,D.NO 23-18-1 B.C ROAD GAJUWAKA VISHAKAPATNAM,Visakhapatnam,9059084901,0.253921
1,83.21495,17.6799066,Lakshmi Medicals and General,D.no.23-175 BC Road Gajuwaka mandal Visakhapatnam,Visakhapatnam,9494401793,0.262890
2,83.2155083,17.6812716,Sri venkateswara medicals and General,D.no.12-60-40 BC Road Gajuwaka mandal Visakhap...,Visakhapatnam,9849716989,0.263911
3,83.2147487,17.6796696,SRI SAI MEDICALS AND GENERAL STORE,D.NO 23-18-2 SHOP NO.4 B.C ROAD GAJUWAKA,Visakhapatnam,9346256179,0.264965
4,83.2154769,17.6820242,SUNEETHA MEDICALS,D.NO 12-60-37/2 SEETHARAM NAGAR B.C ROAD GAJUW...,Visakhapatnam,9848189044,0.269273
...,...,...,...,...,...,...,...
164,83.2381361,17.6895406,SRI VIJAYA DURGA MEDICALS,D.NO 62-1-1 MAIN ROAD SRIHARIPURAM VISHAKAPATNAM,Visakhapatnam,9640891695,2.795672
165,83.1870439,17.6756195,HARSH A TEA MEDICAL AND GENERAL STORE,DOOR NO 27-12-8 SUNDARAYYA COLONY PEDAGANTYAD...,Visakhapatnam,9885180235,2.808210
166,83.1870004,17.6753111,M/s.ADILAKSHMI MEDICAL GENERAL STORES,DO. NO.27-11-77 SUNDARAYYA COLONY GAJUWAKA VIS...,Visakhapatnam,9291604279,2.820613
167,83.240701,17.6831469,SRI SAI SURYA MEDICALS AND GENERAL STORES,D.NO 60-30-3/1 JANTHACOLONY MALKAPURAM GAJUWAK...,Visakhapatnam,9949965265,2.923398
